In [1]:
%pwd
%cd C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price

C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price


In [2]:
pip install pyspark praw nltk pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tweepy praw requests pandas textblob vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "solana": "SOL",
    "sol": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

print("🔄 Starting Reddit post & comment sentiment streaming...")
print("Press Ctrl+C to stop.\n")

try:
    while True:
        records = []

        # Fetch latest posts
        for post in reddit.subreddit("CryptoCurrency").new(limit=10):
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")

            # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue

            # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"{date} | {user_id} | POST | {found_coin} | Sentiment: {sentiment:+.3f}")

            # Fetch top-level comments
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]:  # limit comments per post
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                # Skip deleted or removed comments
                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                # Detect crypto mention in comment
                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break

                # Sentiment for comment
                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])

        # Save to CSV
        if records:
            df = pd.DataFrame(records, columns=[
                "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
            ])
            df.to_csv(CSV_FILE, mode="a", header=False, index=False)
            print(f"✅ Saved {len(records)} new entries to {CSV_FILE}")

        # Wait before next update
        time.sleep(120)  # every 2 minutes

except KeyboardInterrupt:
    print("\n🛑 Stopped streaming by user.")


🔄 Starting Reddit post & comment sentiment streaming...
Press Ctrl+C to stop.



C:\Users\yashs\AppData\Local\Temp\ipykernel_23152\543877309.py:52: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post.created_utc).strftime("%Y-%m-%d %H:%M:%S")


2025-11-15 17:58:58 | KIG45 | POST | BTC | Sentiment: +0.000


C:\Users\yashs\AppData\Local\Temp\ipykernel_23152\543877309.py:74: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")


2025-11-15 17:13:26 | Busy-Pomegranate7551 | POST | ETH | Sentiment: +0.981
2025-11-15 17:00:00 | DryMyBottom | POST | BTC | Sentiment: +0.000
2025-11-15 10:47:26 | TheGreatCryptopo | POST | BTC | Sentiment: -0.681
✅ Saved 36 new entries to reddit_crypto_sentiment.csv
2025-11-15 17:58:58 | KIG45 | POST | BTC | Sentiment: +0.000
2025-11-15 17:13:26 | Busy-Pomegranate7551 | POST | ETH | Sentiment: +0.981
2025-11-15 17:00:00 | DryMyBottom | POST | BTC | Sentiment: +0.000
2025-11-15 10:47:26 | TheGreatCryptopo | POST | BTC | Sentiment: -0.681
✅ Saved 36 new entries to reddit_crypto_sentiment.csv
2025-11-15 17:58:58 | KIG45 | POST | BTC | Sentiment: +0.000
2025-11-15 17:13:26 | Busy-Pomegranate7551 | POST | ETH | Sentiment: +0.981
2025-11-15 17:00:00 | DryMyBottom | POST | BTC | Sentiment: +0.000
2025-11-15 10:47:26 | TheGreatCryptopo | POST | BTC | Sentiment: -0.681
✅ Saved 36 new entries to reddit_crypto_sentiment.csv


In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "solana": "SOL",
    "sol": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)


# --- Define the desired start date: August 1, 2025 ---
start_date_str = "2025-08-01 00:00:00"

# Convert the string date to a datetime object, assuming UTC
start_datetime = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S")

# Convert the datetime object to a Unix timestamp (seconds since epoch)
start_timestamp = start_datetime.timestamp()
post_timestamp = post.created_utc
print("Start timeestamp:",start_timestamp)
print("Post timestamp:",post_timestamp)
print("🔄 Starting Reddit post & comment sentiment streaming...")
print("Press Ctrl+C to stop.\n")

try:
    while True:
        

# Use the 'all' time filter to get posts from a long period, up to PRAW's 1000 post limit.
# Note: This is an approximation and will not guarantee ALL posts from August to today.
# You must use the `top()` or `controversial()` sort, as `new()` doesn't accept time_filter.

        records = []
# Fetch up to 1000 of the top posts since August 1st
        for post in reddit.subreddit("CryptoCurrency").top(time_filter="all", limit=1000):
            post_timestamp = post.created_utc
    
    # Check if the post is within your desired time frame (August 1st to Today)
        if post_timestamp >= start_timestamp:
        # --- Existing Post Processing Logic ---
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")

        # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue
        
        # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"{date} | {user_id} | POST | {found_coin} | Sentiment: {sentiment:+.3f}")
        
        # Fetch top-level comments (Keep this existing logic)
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]: # limit comments per post
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
            
            # Skip deleted or removed comments
                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue
            
            # Detect crypto mention in comment
                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break
            
            # Sentiment for comment
                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])
                # print(f"{comment_date} | {comment_user} | COMMENT | {comment_coin} | Sentiment: {comment_sent:+.3f}") # Optional print for comments

        elif post_timestamp < start_timestamp:
        # Since we are iterating through 'top' posts, we can stop once we hit posts older than our start date
            print("Reached posts older than August 1st. Stopping iteration.")
            break
except KeyboardInterrupt:
    print("\n🛑 Stopped streaming by user.")

    
    # --- End of Post Processing Logic ---

# --- End of Post Fetching Loop ---

Start timeestamp: 1754031600.0
Post timestamp: 1620066692.0
🔄 Starting Reddit post & comment sentiment streaming...
Press Ctrl+C to stop.

Reached posts older than August 1st. Stopping iteration.


In [ ]:
# List of time filters to check, ordered from newest to oldest
TIME_FILTERS = ["day", "week", "month", "year"]

# Define the start date (as before)
start_date_str = "2025-08-15 00:00:00"
start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

# Initialize a set to prevent processing the same post multiple times
processed_ids = set()

In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "solana": "SOL",
    "sol": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

print("🔄 Starting Reddit post & comment sentiment streaming...")

# ----------------------------------------------------------------------
# NEW FETCHING LOGIC
# ----------------------------------------------------------------------

records = []
print(f"Fetching posts since {start_date_str}...")

# 1. Iterate over the time filters
for time_filter in TIME_FILTERS:
    print(f"\n--- Checking Top posts for: {time_filter.upper()} ---")

    # Fetch the top posts for the current time filter (max 1000 per request)
    # The 'limit=None' tells PRAW to fetch up to its internal limit (usually 1000)
    for post in reddit.subreddit("CryptoCurrency").top(time_filter=time_filter, limit=None):
        post_timestamp = post.created_utc

        # Check 1: Has this post been processed already? (Prevents duplicates from different time filters)
        if post.id in processed_ids:
            continue

        # Check 2: Is the post within the required August 1st to Today range?
        if post_timestamp >= start_timestamp:
            
            # Mark the post as processed
            processed_ids.add(post.id)

            # --- Start Existing Processing Logic (Post) ---
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")

            # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            # ... (Rest of your crypto keyword detection logic)
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue
            
            # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"✅ Found Post ({time_filter}): {date} | {found_coin} | Sentiment: {sentiment:+.3f}")

            # --- Start Existing Comment Processing Logic ---
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]:
                # ... (Your existing comment processing and sentiment logic goes here)
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break

                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])
                
        elif time_filter == "year" and post_timestamp < start_timestamp:
            # Only break out of the loop if we're on the longest time filter and found old data.
            # Shorter filters ("day", "week") will naturally stop finding posts as they go too far back.
            print(f"--- Stopping fetch: Hit post older than {start_date_str} on the 'year' filter. ---")
            break

# ----------------------------------------------------------------------
# END NEW FETCHING LOGIC
# ----------------------------------------------------------------------

# --- Final Saving Logic (Run once after all fetching is complete) ---
if records:
    df = pd.DataFrame(records, columns=["date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"])
    df.to_csv(CSV_FILE, mode="a", header=False, index=False)
    print(f"\n✅ FINAL SAVE: Collected and saved {len(records)} unique entries to {CSV_FILE}")

print("Processing complete.")

🔄 Starting Reddit post & comment sentiment streaming...
Fetching posts since 2025-09-15 00:00:00...

--- Checking Top posts for: DAY ---


C:\Users\yashs\AppData\Local\Temp\ipykernel_31020\2245392545.py:72: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")


✅ Found Post (day): 2025-11-14 12:15:45 | BTC | Sentiment: -0.421


C:\Users\yashs\AppData\Local\Temp\ipykernel_31020\2245392545.py:96: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")


✅ Found Post (day): 2025-11-14 20:08:29 | BTC | Sentiment: -0.670
✅ Found Post (day): 2025-11-14 17:06:34 | BTC | Sentiment: -0.421
✅ Found Post (day): 2025-11-15 04:51:36 | BTC | Sentiment: -0.296
✅ Found Post (day): 2025-11-14 12:55:28 | ETH | Sentiment: +0.402
✅ Found Post (day): 2025-11-15 02:50:11 | ETH | Sentiment: +0.000
✅ Found Post (day): 2025-11-15 07:33:39 | BTC | Sentiment: +0.000
✅ Found Post (day): 2025-11-14 16:31:37 | BTC | Sentiment: +0.984
✅ Found Post (day): 2025-11-14 14:19:40 | BTC | Sentiment: +0.440
✅ Found Post (day): 2025-11-15 01:00:46 | ETH | Sentiment: +0.954
✅ Found Post (day): 2025-11-14 20:02:44 | ETH | Sentiment: +0.788
✅ Found Post (day): 2025-11-15 07:45:47 | BTC | Sentiment: -0.186
✅ Found Post (day): 2025-11-14 21:43:38 | ETH | Sentiment: +0.000
✅ Found Post (day): 2025-11-14 16:29:24 | BTC | Sentiment: +0.975
✅ Found Post (day): 2025-11-14 21:24:13 | BTC | Sentiment: +0.960
✅ Found Post (day): 2025-11-15 06:06:59 | ETH | Sentiment: +0.976
✅ Found Po

In [ ]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# create spark context and spark session
from pyspark import SparkConf, SparkContext


In [ ]:
%pwd

%cd C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price

C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price


In [ ]:
conf = SparkConf().setMaster("local").setAppName("MIS584_Project")

In [ ]:
import os
# Replace this with the actual path to your JDK 8 or JDK 11 installation
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_202" 

# Then re-run your findspark and SparkContext setup
import findspark
findspark.init('C:\\spark') 
from pyspark import SparkConf, SparkContext
# (Assuming you define 'conf' here)
#conf = SparkConf().setAppName("SparkSQL_Project").setMaster("local[*]")

# This should now succeed:
sc = SparkContext(conf = conf)

In [ ]:
import os
from pyspark.sql import SparkSession

# --- 1. SET JAVA_HOME AND SPARK_HOME ---
# Set JAVA_HOME to the path you provided.
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_202"

# You also need to ensure SPARK_HOME is set correctly to your Spark installation root.
# Replace this with your actual Spark path (e.g., C:\spark\spark-3.5.0-bin-hadoop3)
os.environ["SPARK_HOME"] = "C:\\spark" 

# --- 2. Initialize Spark Session ---
try:
    spark = (SparkSession.builder
             .appName("MIS584_Project")
             .getOrCreate())
    
    print("✅ Spark Session created successfully!")

except Exception as e:
    # If it still fails, the error will be clearer (e.g., Py4JJavaError with a root cause)
    print(f"❌ Failed to create Spark Session. Error: {e}")

❌ Failed to create Spark Session. Error: 'JavaPackage' object is not callable


In [ ]:
os.environ["SPARK_HOME"] = "C:\\spark"

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MIS584_Project").getOrCreate()

TypeError: 'JavaPackage' object is not callable

In [2]:
sc.stop()

NameError: name 'sc' is not defined